In [1]:
import pandas as pd
import numpy as np
import datetime

import matplotlib
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px


import os
#pip install chart_studio

pd.set_option('display.max_columns', None)
#pd.options.plotting.backend = "plotly"

In [2]:
os.getcwd()

'D:\\Google Drive\\Masters\\Sem 3\\Python_Stuff\\Aaron [No Ship]'

In [3]:
full_path = r"D:\Google Drive\Masters\Sem 3\EBAC No Ship, Sherlock\Capstone\raw data"
os.chdir(full_path)
#os.chdir(r"C:\Users\Admin\Google Drive\Masters\Sem 3\EBAC No Ship, Sherlock\Capstone\raw data")

In [4]:
os.getcwd()

'D:\\Google Drive\\Masters\\Sem 3\\EBAC No Ship, Sherlock\\Capstone\\raw data'

# Get Masterdata of shipment routes

In [5]:
# Want to get the shipment data
excel_route_file = "APJ Route TAT Master File V20210129.xlsx"


In [6]:
def get_shipment_routes(excel_route_file):
    xls = pd.ExcelFile(excel_route_file)
    df_ship_routes = pd.read_excel(xls,"MasterData" )
    xls = [] # Free up resource
    print("Length of Excel: {}".format(len(df_ship_routes)))
    df_ship_routes.head()
    # Check that there are no duplicates
    sum(df_ship_routes['Route'].duplicated())
    #df_ship_routes = df_ship_routes['Mode'].str.lower() == "air"
    print("The count of mode by air: {}".format(sum(df_ship_routes['Mode'].str.lower() == "ocean")))
    return df_ship_routes

In [7]:
df_ship_routes = get_shipment_routes(excel_route_file)

Length of Excel: 11958
The count of mode by air: 0


In [8]:
len(df_ship_routes)


11958

# Individual Shipment Data

In [9]:
os.chdir(r".\monthly")
file_names = [f for f in os.listdir() if f.find("SearchResult")>=0 ]

print(file_names)

['SearchResult_1615300381378_Jan21.csv', 'SearchResult_1615301179189_Dec20.csv', 'SearchResult_1615301591520_Nov20.csv', 'SearchResult_1615301861292_Oct20.csv', 'SearchResult_1615302057067_Sep20.csv', 'SearchResult_1615302586308_Aug20.csv', 'SearchResult_1615302848713_Jul20.csv', 'SearchResult_1615303098163_Jun20.csv', 'SearchResult_1615303294492_May20.csv', 'SearchResult_1615303594160_Apr20.csv', 'SearchResult_1615303885072_Mar20.csv', 'SearchResult_1615304080226_Feb20.csv']


In [10]:


def get_shipment_data(excel_ship_file):
    print("Excel File Name: {}".format(excel_ship_file))
    df_ship_data = pd.read_csv(excel_ship_file)
    print("File shape is: {}".format(df_ship_data.shape))
    return df_ship_data



In [50]:

# Takes a df and 2 of it's columns. If the original is null, replaces it with the replacement. 
def replace_original_if_null(df, original, replacement):
    df[original] = df.apply(
        lambda row: row[replacement] if pd.isnull(row[original]) else row[original],
        axis=1)
    return df

def wrongyear_data_cleaning(df):
        # ---------------- Correction of timing for 2020/2021/2022 -------------------------
    
    #Change Arrived at destination port year from 2020 to 2021 when ATA time - ATA received time < -300 
    df['Arrived at Destination Port Milestone Time'] = df['Arrived at Destination Port Milestone Time'].mask(
        (df['Arrived at Destination Port Milestone Time']- \
                         df['Arrived at Destination Port Milestone Received Time']).dt.days < -300, 
        df['Arrived at Destination Port Milestone Time'] +  pd.to_timedelta(366, unit='D'))
    
    
    df['Arrived at Destination Port Milestone Local Time'] = df['Arrived at Destination Port Milestone Local Time'].mask(
        (df['Arrived at Destination Port Milestone Local Time'] - \
             df['Arrived at Destination Port Milestone Time']).dt.days < -300,
            df['Arrived at Destination Port Milestone Local Time'] +  pd.to_timedelta(366, unit='D'))
    
    #Change Arrived at destination port year from 2022 to 2021 when ATA time - ATA received time > 300 
    df['Arrived at Destination Port Milestone Time'] = df['Arrived at Destination Port Milestone Time'].mask(
        (df['Arrived at Destination Port Milestone Time']- \
                         df['Arrived at Destination Port Milestone Received Time']).dt.days >= 300, 
        df['Arrived at Destination Port Milestone Time'] -  pd.to_timedelta(366, unit='D'))
    
    
    df['Arrived at Destination Port Milestone Local Time'] = df['Arrived at Destination Port Milestone Local Time'].mask(
        (df['Arrived at Destination Port Milestone Local Time'] - \
         df['Arrived at Destination Port Milestone Time']).dt.days > 300, 
        df['Arrived at Destination Port Milestone Local Time'] -  pd.to_timedelta(366, unit='D'))



    #Change Departed from Origin Port Milestone year from 2020 to 2021 when ATD time - ATD received time < -300 
    df['Departed from Origin Port Milestone Time'] = df['Departed from Origin Port Milestone Time'].mask(
         (df['Arrived at Destination Port Milestone Local Time'] - \
         df['Arrived at Destination Port Milestone Time']).dt.days < -300, 
        df['Departed from Origin Port Milestone Time'] +  pd.to_timedelta(366, unit='D'))
    
    df['Departed from Origin Port Milestone Local Time'] = df['Departed from Origin Port Milestone Local Time'].mask(
        (df['Departed from Origin Port Milestone Local Time'] - \
         df['Departed from Origin Port Milestone Time']).dt.days < -300, 
        df['Departed from Origin Port Milestone Local Time'] +  pd.to_timedelta(366, unit='D'))
    return df



def data_cleaning(df):
    
    # First, Want to clean up data that is obviously duplicated. 
    print("Data that is duplicated exactly: {}".format(sum(df.duplicated())))
    df = df[~df.duplicated()]

    # If H-K missing, replace with Latest D-G
    #[(Replacement, original)]
    replace_column = [('Onboard at Origin Port Milestone Local Time','Departed from Origin Port Milestone Local Time'),
       ('Onboard at Origin Port Milestone Time','Departed from Origin Port Milestone Time'),
       ('Onboard at Origin Port Milestone Received Time','Departed from Origin Port Milestone Received Time'),
       ('Onboard at Origin Port Milestone Source', 'Departed from Origin Port Milestone Source')]
    
    for (r,o) in replace_column:
        df =  replace_original_if_null(df=df, original=o, replacement=r)
        
    
    # Cast to datetime. 
    col_to_datetime = ['Departed from Origin Port Milestone Local Time',
       'Departed from Origin Port Milestone Time',
       'Departed from Origin Port Milestone Received Time',
       'Arrived at Destination Port Milestone Local Time',
       'Arrived at Destination Port Milestone Time',
       'Arrived at Destination Port Milestone Received Time',
       'Est. Arrival at Destination Port Milestone Local Time',
       'Est. Arrival at Destination Port Milestone Time',
       'Est. Arrival at Destination Port Milestone Received Time']

    for col in col_to_datetime:
        #df = df[df[c].notnull()] # Want to get rid of all the null values. 
        
        print("Changing Column '{}' to datetime.".format(col))
        df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')
        
        #df[c] = df[c].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S') if not pd.isnull(x) else x)
        
    
    # Want to do the year clean up for those data with wrongly labelled years. 
    df = wrongyear_data_cleaning(df)
        
    3# M later than I
    #"Arrived at Destination Port Milestone Time" > "Departed from Origin Port Milestone Time"
    df = df[df["Arrived at Destination Port Milestone Time"] > df["Departed from Origin Port Milestone Time"]]

    ## R Before M & N
    #"Est. Arrival at Destination Port Milestone Received Time" < "Arrived at Destination Port Milestone Time"
    df = df[df["Est. Arrival at Destination Port Milestone Received Time"] < df["Arrived at Destination Port Milestone Time"]]

    #"Est. Arrival at Destination Port Milestone Received Time" < "Arrived at Destination Port Milestone Received Time"
    df=df[df["Est. Arrival at Destination Port Milestone Received Time"] < df["Arrived at Destination Port Milestone Received Time"]]

     ## R Later than I
     #"Est. Arrival at Destination Port Milestone Received Time" > "Departed from Origin Port Milestone Time"
    df=df[df["Est. Arrival at Destination Port Milestone Received Time"] < df["Departed from Origin Port Milestone Time"]]

     # Sort by Latest M, Take earliest R
     # "Arrived at Destination Port Milestone Time"
     # "Est. Arrival at Destination Port Milestone Received Time"
    df.sort_values(["Arrived at Destination Port Milestone Time", "Est. Arrival at Destination Port Milestone Received Time"], ascending=[False, True], inplace=True)

    # Drop the duplicated Shipment IDs.
    df.drop_duplicates(subset=['Shipment id'], keep='first')

    return df







In [46]:
# To read all the excel sheets. 
df_unique_shipment_all = pd.DataFrame()
for f in file_names:
    df_unique_shipment_all=df_unique_shipment_all.append(get_shipment_data(f))

Excel File Name: SearchResult_1615300381378_Jan21.csv


c:\users\risk\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning:

Columns (21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.



File shape is: (164584, 33)
Excel File Name: SearchResult_1615301179189_Dec20.csv
File shape is: (148071, 33)
Excel File Name: SearchResult_1615301591520_Nov20.csv


c:\users\risk\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning:

Columns (21,23) have mixed types.Specify dtype option on import or set low_memory=False.



File shape is: (105069, 33)
Excel File Name: SearchResult_1615301861292_Oct20.csv
File shape is: (124359, 33)
Excel File Name: SearchResult_1615302057067_Sep20.csv
File shape is: (600000, 33)
Excel File Name: SearchResult_1615302586308_Aug20.csv
File shape is: (600000, 33)
Excel File Name: SearchResult_1615302848713_Jul20.csv
File shape is: (600000, 33)
Excel File Name: SearchResult_1615303098163_Jun20.csv
File shape is: (119388, 33)
Excel File Name: SearchResult_1615303294492_May20.csv
File shape is: (600000, 33)
Excel File Name: SearchResult_1615303594160_Apr20.csv
File shape is: (600000, 33)
Excel File Name: SearchResult_1615303885072_Mar20.csv
File shape is: (276524, 33)
Excel File Name: SearchResult_1615304080226_Feb20.csv
File shape is: (105049, 33)


In [47]:
df_unique_shipment_original = data_cleaning(df_unique_shipment_all)

Data that is duplicated exactly: 2121838
Changing Column 'Departed from Origin Port Milestone Local Time' to datetime.
Changing Column 'Departed from Origin Port Milestone Time' to datetime.
Changing Column 'Departed from Origin Port Milestone Received Time' to datetime.
Changing Column 'Arrived at Destination Port Milestone Local Time' to datetime.
Changing Column 'Arrived at Destination Port Milestone Time' to datetime.
Changing Column 'Arrived at Destination Port Milestone Received Time' to datetime.
Changing Column 'Est. Arrival at Destination Port Milestone Local Time' to datetime.
Changing Column 'Est. Arrival at Destination Port Milestone Time' to datetime.
Changing Column 'Est. Arrival at Destination Port Milestone Received Time' to datetime.


ValueError: keep must be either "first", "last" or False

In [ ]:
df_unique_shipment = df_unique_shipment_original.copy()

In [ ]:
df_unique_shipment.head()

In [ ]:
len(df_unique_shipment)

In [ ]:
len(df_unique_shipment["Shipment id"].unique())

In [ ]:
# Want the First ones, since we have sorted it already

In [ ]:
#df_unique_shipment=df_unique_shipment[~ df_unique_shipment["Shipment id"].duplicated(keep="first")]
df_unique_shipment.drop_duplicates(subset ="Shipment id", keep = "first", inplace = True)

In [ ]:
len(df_unique_shipment)

##### Might want to looking into correcting data entry errors with the years. 

In [ ]:
# df=df_unique_shipment

In [ ]:
# time_diff = abs(df["Arrived at Destination Port Milestone Time"] - df["Departed from Origin Port Milestone Time"])
# time_diff > pd.Timedelta("300 days")

In [ ]:
# df[(df["Est. Arrival at Destination Port Milestone Received Time"] - df["Arrived at Destination Port Milestone Time"]) > pd.Timedelta("300 days")][["Est. Arrival at Destination Port Milestone Received Time","Arrived at Destination Port Milestone Time"]]

In [ ]:
len(df_unique_shipment)

In [ ]:
df_unique_shipment["Delay Est/Arrival Duration"] = df_unique_shipment["Arrived at Destination Port Milestone Time"] - df_unique_shipment["Est. Arrival at Destination Port Milestone Time"]

In [ ]:
# Want to do a little clean up here as there are ships that are earlier by 100 days
df_unique_shipment=df_unique_shipment[df_unique_shipment["Delay Est/Arrival Duration"] > -pd.Timedelta("100 days")]

In [ ]:
df_unique_shipment[df_unique_shipment["Delay Est/Arrival Duration"]<pd.Timedelta("0 days")].sort_values("Delay Est/Arrival Duration")["Delay Est/Arrival Duration"]

In [ ]:
len(df_unique_shipment)

In [ ]:
df_unique_shipment[df_unique_shipment["Delay Est/Arrival Duration"] < pd.Timedelta("0 days")].tail()

In [ ]:
df_unique_shipment.fillna({"Origin Ctry":"-"}, inplace=True)

In [ ]:
# df_ship_data[df_ship_data.duplicated(subset=["Shipment id"])]["Shipment id"].value_counts().nlargest(1)

In [ ]:
# df_ship_data[df_ship_data["Shipment id"] == 39640134][~ df_ship_data[df_ship_data["Shipment id"] == 39640134].duplicated()].to_excel("duplicate.xlsx")

In [ ]:
matplotlib.rcParams['figure.figsize'] = [15, 5]

In [ ]:
#df_unique_shipment["Route"].value_counts().iplot(kind='bar')
#df_unique_shipment["Route"].value_counts().nlargest(50).plot(kind="bar")

In [ ]:
# fig = px.histogram(df_unique_shipment["Route"]).update_xaxes(categoryorder="total descending")
# fig.show()

In [ ]:
# Want to get a list of unique shipment
#unique_shipment_ID = df_ship_data["Shipment id"].unique().tolist()

# Want to get the columns that are at least 80% Filled

In [ ]:
def get_col_names(df, col_filled=80):

    # Want to get the column with the most missing data
    missing_value_series = df.isnull().sum() / df.shape[0] * 100.00
    # Sort the missing %
    missing_value_series.sort_values(ascending=False, inplace=True)
    #missing_value_series
    col_dict=missing_value_series.to_dict()
    col = [k for k, d in col_dict.items() if d < (100 - col_filled)]
    return col

In [ ]:
df_unique_shipment["Route"] = df_unique_shipment["Route"].astype("str")
df_ship_routes['Route']  = df_ship_routes['Route'].astype("str")

In [ ]:
# Checking if the data is of string type
#type(df_unique_shipment['Route'])
# type(df_ship_routes['Route'][0])

# Combined df

In [ ]:
df_combined = df_unique_shipment.merge(df_ship_routes, on="Route", how='left')

In [ ]:
list(df_combined.columns) 

In [ ]:
df_combined=df_combined[get_col_names(df_combined)]

In [ ]:
df_combined.head()

In [ ]:
fig = px.histogram(df_combined["Destination Ctry"], title='Histogram of Destination Ctry').update_xaxes(categoryorder="total descending")
fig.show()

In [ ]:
fig = px.histogram(df_combined["Origin Ctry"], title='Histogram of Origin Ctry').update_xaxes(categoryorder="total descending", title="Origin Country")
fig.show()

In [ ]:
df_combined[df_combined["Description"].isnull()]["Route"].unique()

In [ ]:
df_combined[["Delay Est/Arrival Duration", "Arrived at Destination Port Milestone Time", "Est. Arrival at Destination Port Milestone Time"]].head()

In [ ]:
# df_combined.iloc[5][["Delay Est/Arrival Duration", "Arrived at Destination Port Milestone Time", "Est. Arrival at Destination Port Milestone Time"]]

In [ ]:
df_combined["Delay Est/Arrival Duration (Hour)"] = df_combined["Delay Est/Arrival Duration"].apply(lambda x: x.total_seconds()/(60*60))

In [ ]:
df_combined["Delay Est/Arrival Duration (Hour)"]

In [ ]:
fig = px.histogram(df_combined[df_combined["Origin Ctry"].notnull()][["Delay Est/Arrival Duration (Hour)", "Origin Ctry"]], title='Delay between estimates', color="Origin Ctry").update_xaxes(categoryorder="total descending", title="Time (Hours)")
fig.show()

In [ ]:
len(df_combined)

In [ ]:
df_combined

In [ ]:
os.chdir("D:\Google Drive\Masters\Sem 3\Python_Stuff\Aaron [No Ship]")
df_combined.to_csv("Cleaned_Data.csv")

In [ ]:
full_path

In [ ]:
"Port of LoadingDestination CtryMode" in df_combined.columns